# Transformer
前面学习了RNN和LSTM以及Seq2Seq，同时也接触了不同类型的注意力，现在要学习一个新的架构，它是后面要学习的Bert和GPT的核心-Transformer。  
Transformer的核心就是自注意力机制，它能够为输入序列中的每个元素分配不同的权重，从而更好地捕捉序列内部的依赖关系。同时其抛弃了RNN和LSTM中的循环结构，采用了全新的编码器-解码器架构，使得模型可以并行处理输入数据，进一步加速训练过程，提高计算效率。  
Transformer架构图:  
<img src="./images/Transformer架构图.png" alt="examples" style="zoom:45%;" />

## Transformer组件
根据上面的架构图，逐个组建去实现。  
1.多头自注意力：通过ScaledDotProductAttention类实现缩放点积注意力机制，然后通过MultiHeadAttention类实现多头自注意力机制  
2.逐位置前馈网络：通过PoswiseFeedForwardNet类实现逐位置前馈网络  
3.正弦位置编码表：通过get_sin_code_table函数生成正弦位置编码表  
4.填充掩码：通过get_attn_pad_mask函数为填充令牌`<pad>`生成注意力掩码，避免注意力机制关注无用的信息  
5.编码器层：通过EncoderLayer类定义编码器的单层  
6.编码器：通过Encoder类定义Transformer完整的编码器部分  
7.后续掩码：通过get_attn_subsequent_mask函数为后续令牌(当前位置后面的信息)生成注意力掩码，避免解码器中的注意力机制"偷窥"未来的目标数据  
8.解码器层：通过DecoderLayer类定义解码器的单层  
9.解码器：通过Decoder类定义Transformer完整的解码器部分  
10.Transformer类：此类将编码器和解码器整合为完整的Transformer模型

### 组件1 多头自注意力(包含残差连接和层归一化)
首先实现Transformer的核心组件，多头自注意力。主要有两个子组件，缩放点积注意力类和多头自注意力类

#### 缩放点积注意力(加入掩码机制)

<img src="./images/scaled_dot_attention_mask.png" alt="examples" style="zoom:45%;" />

掩码机制的说明:  
这里加入的掩码机制主要是填充注意力掩码：当处理变长序列时，通常会对较短序列进行填充，使所有序列的长度一致，一边进行批量处理。  
但是填充值是没有意义的，所以需要通过掩码机制将其设置为极小值，这样在应用softmax时填充位置的权重将接近于0，不会对模型产生影响。 

In [71]:
import numpy as np
import torch
import torch.nn as nn
d_k = 64 
d_v = 64

# 定义缩放点积注意力函数
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()
    def forward(self, Q, K, V, attn_mask):
        #------------------------- 维度信息 --------------------------------
        # Q K V [batch_size, n_heads, seq_len, seq_dim] -> fea_dim = n_heads * seq_dim
        # attn_mask [batch_size, n_heads, len_q, len_k]
        #------------------------------------------------------------------
        
        # 计算注意力分数（原始权重）[batch_size, n_heads, len_q, len_k]
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)
        
        # 使用注意力掩码，将attn_mask中值为1(或者是布尔类型的true)的位置(填充位置)的权重替换为极小值
        scores.masked_fill_(attn_mask, -1e9)
        
        # 对注意力分数进行softmax归一化处理
        # weight -> [batch_size, n_heads, len_q, len_k]
        weight = nn.Softmax(dim=-1)(scores)
        
        # 再次点积操作，计算上下文向量(注意力的输出)
        # context -> [batch_size, n_heads, len_q, dim_v]
        context = torch.matmul(weight, V)
        return context, weight      

#### 多头自注意力类(残差连接和层归一化)

<img src="./images/multi_head_attention.png" alt="examples" style="zoom:45%;" />

残差连接:  
残差连接是一种在神经网络中广泛使用的技术，用于加快网络的训练和提高模型的性能。在残差网络中，每个层都添加了一个跨层连接，可以将输入数据直接连接到输出数据，也可以将输入数据直接传传递到后续层次，从而提高信息的传递效率和网络的训练速度。  
其实就是将输入与输出直接相加然后再通过激活函数。

层归一化:  
层归一化是一种正则化技巧，用于缓解神经网络中的内部协变量偏移问题。在Transformer模型中，层归一化通常应用于残差连接之后，用于对输出进行归一化。

ref:  
https://zhuanlan.zhihu.com/p/620297938  
https://zhuanlan.zhihu.com/p/353062927

In [72]:
# 定义多头自注意力类
d_embedding = 512 # Embedding维度
n_heads = 8 # 多头注意力的头数 fea_dim = 512 / 8 = 64
batch_size = 3
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_embedding, d_k * n_heads) # Q的线性变换层
        self.W_K = nn.Linear(d_embedding, d_k * n_heads) # K的线性变换层
        self.W_V= nn.Linear(d_embedding, d_v * n_heads) # V的线性变换层
        self.linear = nn.Linear(d_v * n_heads, d_embedding)
        self.layer_norm = nn.LayerNorm(d_embedding)
    
    def forward(self, Q, K, V, attn_mask):
        #------------------------- 维度信息 --------------------------------
        # Q K V -> [batch_size, len_q/len_k,len_v, embedding_dim]
        #------------------------------------------------------------------
        residual, batch_size = Q, Q.size(0) # 保留残差连接
        
        # 将输入进行线性变换和重塑，以便后续处理
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1, 2)
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1, 2)
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1, 2)
        
        #------------------------- 维度信息 -------------------------------- 
        # q_s k_s v_s -> [batch_size, n_heads, len_q/k/v, d_k/k/v]
        #------------------------------------------------------------------
        # 将注意力掩码复制到多头
        # 先通过unsqueeze来增加一个维度(batch_size, 1, len_q, len_k)
        # 然后复制n_heads次数据，保证在每个头上的掩码数据都一致
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)
        
        #------------------------- 维度信息 -------------------------------- 
        # attn_mask -> [batch_size, n_heads, len_q, len_k]
        #------------------------------------------------------------------
        # 使用缩放点积注意力计算上下文和注意力权重
        context, weights = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        
        #------------------------- 维度信息 -------------------------------- 
        # context -> [batch_size, n_heads, len_q, dim_v]
        # weights -> [batch_size, n_heads, len_q, len_k]
        #------------------------------------------------------------------
        # 将多个头的上下文向量连接在一起
        # context -> [batch_size, len_q, n_heads * dim_v]
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v)
        
        # 线性变换，转换为embedding_size
        output = self.linear(context)
        #------------------------- 维度信息 -------------------------------- 
        # output -> [batch_size, len_q, embedding_dim]
        #------------------------------------------------------------------

        # 与输入残差连接，并进行层一化处理
        output = self.layer_norm(output + residual)
        
        # 返回层归一化的结果和注意力权重
        return output, weights        

### 组件2 逐位置前馈网络
在编码器和解码器的每一层注意力层之后都会有一个前馈网络，起到进一步提取特征和表示的作用。论文中为全联接层，只不过这里不需要对注意力层的输出做flatten操作(堆叠)，所以这个前馈神经网络独立地作用在每输入序列的每个位置(token)上，这里使用了一维卷积层作为前馈神经网络。

In [73]:
# 定义逐位置前馈网络
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, d_ff=2048):
        super(PoswiseFeedForwardNet, self).__init__()
        # 定义一维卷积层，用于将输入映射到高维度
        self.conv1 = nn.Conv1d(in_channels=d_embedding, out_channels=d_ff, kernel_size=1)
        # 定义一维卷积层，用于将输入映射回原始维度
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_embedding, kernel_size=1)
        # 定义层归一化
        self.layer_norm = nn.LayerNorm(d_embedding)
    def forward(self, inputs):
        #------------------------- 维度信息 -------------------------------- 
        # input -> [batch_size, len_q, embedding_dim]
        #------------------------------------------------------------------
        residual = inputs # 保留残差连接
        
        # 将输入进行卷积运算之后的激活函数时ReLU
        output = nn.ReLU()(self.conv1(inputs.transpose(1, 2)))
        
        #------------------------- 维度信息 -------------------------------- 
        # output -> [batch_size, d_ff, len_q]
        #------------------------------------------------------------------
        residual = inputs # 保留残差连接
        
        # 使用第二个卷积层机进行降维
        output = self.conv2(output).transpose(1, 2)
        #------------------------- 维度信息 -------------------------------- 
        # output -> [batch_size, len_q, d_embedding]
        #------------------------------------------------------------------
        
        # 残差连接+归一化
        output = self.layer_norm(output + residual)
        
        return output    

### 组件3 正弦位置编码表
在RNN中是不需要位置编码的，因为在建模过程中天然具有顺序的概念。但是Transformer架构使用多头自注意力的并行计算方式，因此丧失了token间的位置信息，所有通过正弦位置编码来引入位置信息。  

ref:  
https://zhuanlan.zhihu.com/p/626828066


In [74]:
# 生成正弦位置编码表的函数，用于在 Transformer 中引入位置信息
def get_sin_enc_table(n_position, embedding_dim):
    #------------------------- 维度信息 --------------------------------
    # n_position: 输入序列的最大长度
    # embedding_dim: 词嵌入向量的维度
    #-----------------------------------------------------------------    
    # 根据位置和维度信息，初始化正弦位置编码表
    sinusoid_table = np.zeros((n_position, embedding_dim))    
    # 遍历所有位置和维度，计算角度值
    for pos_i in range(n_position):
        for hid_j in range(embedding_dim):
            angle = pos_i / np.power(10000, 2 * (hid_j // 2) / embedding_dim)
            sinusoid_table[pos_i, hid_j] = angle    
    # 计算正弦和余弦值
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i 偶数维
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1 奇数维    
    #------------------------- 维度信息 --------------------------------
    # sinusoid_table 的维度是 [n_position, embedding_dim]
    #----------------------------------------------------------------   
    return torch.FloatTensor(sinusoid_table)  # 返回正弦位置编码表

### 组件4 填充掩码
在nlp任务中序列的长度通常时不固定的，因此为了实现批量处理，通常需要进行填充操作(PAD)到固定的长度，也就是使用`<pad>`符号来作为填充符(token的值为0)。  
但是这些填充信息是没有任何意义的，所以希望模型在计算注意力时忽略这部分数据。因此需要使用填充掩码机制，将无用的信息屏蔽，防止模型在计算注意时关注到填充位。  

<img src="./images/pad_mask.png" alt="examples" style="zoom:45%;" />

In [75]:
def get_attn_pad_mask(seq_q, seq_k):
    #------------------------- 维度信息 --------------------------------
    # seq_q 的维度是 [batch_size, len_q]
    # seq_k 的维度是 [batch_size, len_k]
    #-----------------------------------------------------------------

    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    
    # 生成boolean类型张量，即将seq_k中token为0的位置变为true
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)
    
    #------------------------- 维度信息 --------------------------------
    # pad_attn_mask 的维度是 [batch_size, 1, len_k]
    #-----------------------------------------------------------------
    # 再次变形为与注意力分数相同形状的张量 -> [batch_size, len_q, len_k]
    pad_attn_mask = pad_attn_mask.expand(batch_size, len_q, len_k)
    
    return pad_attn_mask
    

### 组件5 编码器层
在构造完上面袋编码器的多个基本组件后，现在来构建一个基本的编码器层

In [76]:
import torch.nn as nn
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention() # 定义多头自注意力层
        self.pos_ffn = PoswiseFeedForwardNet() # 定义逐位置前馈网络
    
    def forward(self, enc_inputs, enc_self_attn_mask):
        #------------------------- 维度信息 --------------------------------
        # enc_inputs 的维度是 [batch_size, seq_len, embedding_dim]
        # enc_self_attn_mask 的维度是 [batch_size, seq_len, seq_len]
        #-----------------------------------------------------------------
        
        # 将编码器输入喂入多头自注意力层，通过linear得到QKV，然后得到注意力的输出和权重
        # enc_outputs -> [batch_size, seq_len, embedding_dim]
        # attn_weights -> [batch_size, n_heads, seq_len, seq_len]
        enc_outputs, attn_weights = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)
        
        # 将多头自注意力的输出输入前馈网络层
        enc_outputs = self.pos_ffn(enc_outputs)
        
        return enc_outputs, attn_weights

### 组件6 编码器
编码器通常是由多个上面的编码器层堆叠而成，目的是为了提取序列中更丰富的特征  
nn.Embedding的说明：weight矩阵(lookup-table)这个矩阵的每一行就是字典中对应索引值的词嵌入(词向量),从形状来看：  
(1 ✖ ️4 one-hot编码) * (4 ✖️ 3 weight) -> 1 ✖️ 3
<img src="./images/embedding.png" alt="examples" style="zoom:50%;" />

from_pretrained()可以指定权重向量
<img src="./images/embedding_pretrained.png" alt="examples" style="zoom:50%;" />

In [77]:
n_layers = 6 # 设置Encoder的层数
class Encoder(nn.Module):
    def __init__(self, corpus):
        super(Encoder, self).__init__()
        # 词嵌入层
        self.src_emb = nn.Embedding(len(corpus.src_vocab), d_embedding)
        # 位置嵌入层
        self.pos_emb = nn.Embedding.from_pretrained(\
            get_sin_enc_table(corpus.src_len+1, d_embedding), freeze=True
        )
        # 编码器层数
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
    
    def forward(self, enc_inputs):
        #------------------------- 维度信息 ---------------------------------
        # enc_inputs 的维度是 [batch_size, source_len] 原始语料序列的长度
        #-----------------------------------------------------------------
        # 创建一个从1到source_len的位置索引序列
        # pos_indices -> [1, source_len]
        pos_indices = torch.arange(1, enc_inputs.size(1) + 1).unsqueeze(0).to(enc_inputs)
        
        # 对输入进行词嵌入并和位置嵌入相加
        # enc_outputs -> [batch_size, seq_len, embedding_dim]
        enc_outputs = self.src_emb(enc_inputs) + self.pos_emb(pos_indices)
        
        # 生成自注意力掩码
        # enc_self_attn_mask -> [batch_size, len_q, len_k] 等同于注意力分数的形状 
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs)
        
        enc_self_attn_weights = [] # 初始化注意力权重，主要适用于保存
        # 将词嵌入向量输入编码器
        # enc_outputs -> [batch_size, seq_len, embedding_dim]
        # enc_self_attn_weights 是一个列表，每个元素的维度是 [batch_size, n_heads, seq_len, seq_len]
        for layer in self.layers:
            enc_outputs, enc_self_attn_weight = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attn_weights.append(enc_self_attn_weight)
        
        return enc_outputs, enc_self_attn_weights

### 组件7 后续掩码
上面完成了编码器，在解码器中，基本组件与编码器类似，唯一不同的是解码器的输入在需要填充掩码袋同时还需要后续掩码。这与解码器的训练是教师强制有关，在训练过程中将真实的输出作为下一个时间步的输入，因此为了模型在预测当前位置时不会关注到未来的信息，使用了后续掩码，其本质是一个上三角矩阵  

<img src="./images/后续掩码.png" alt="examples" style="zoom:50%;" /> 


In [78]:
import numpy as np
def get_attn_subsequent_mask(seq):
    #------------------------- 维度信息 --------------------------------
    # seq 的维度是 [batch_size, seq_len(Q)=seq_len(K)]
    #-----------------------------------------------------------------
    # 获取输入序列的形状
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    
    # 通过numpy创建一个上三角矩阵(triu = triangle upper)
    # subsequent_mask -> [batch_size, seq_len(Q), seq_len(K)]
    subsequent_mask = np.triu(np.ones(attn_shape), k=1)
    
    # 将numpy数组转换为Pytorch张量，并将数据类型设置为byte(布尔值)
    subsequent_mask = torch.from_numpy(subsequent_mask).byte()
    
    return subsequent_mask

### 组件8 解码器层
解码器层与编码器层的组成类似，只是多了后续掩码以及编码器-解码器注意力

In [79]:
# 定义解码器层类
class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention() # 定义多头自注意力层
        self.dec_enc_attn = MultiHeadAttention() # 定义编码器-解码器注意力层
        self.pos_fn = PoswiseFeedForwardNet() # 位置前馈神经网络层
    
    def forward(self, dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        #------------------------- 维度信息 --------------------------------
        # dec_self_attn_mask 多头自注意力的掩码(填充掩码+后续掩码)
        # dec_enc_attn_mask 编码器-解码器注意力掩码(填充掩码)
        # dec_inputs 的维度是 [batch_size, target_len, embedding_dim]
        # enc_outputs 的维度是 [batch_size, source_len, embedding_dim]
        # dec_self_attn_mask 的维度是 [batch_size, target_len, target_len]
        # dec_enc_attn_mask 的维度是 [batch_size, target_len, source_len]
        #-----------------------------------------------------------------
        
        # 多头自注意力层 (Q K V mask)
        # dec_outputs -> [batch_size, target_len, embedding_dim]
        # dec_self_attn -> [batch_size, n_heads, target_len, target_len]
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)
        
        # 编码器-解码器注意力层
        # dec_outputs -> [batch_size, target_len, embedding_dim]
        # dec_self_attn -> [batch_size, n_heads, target_len, source_len]
        dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        
        # 输入位置前馈网络层
        dec_outputs = self.pos_fn(dec_outputs)
        
        return dec_outputs, dec_self_attn, dec_enc_attn 

### 组件9 解码器
解码器与编码器类似，也是由多个解码器层堆叠而层的

In [80]:
n_layers = 6
class Decoder(nn.Module):
    def __init__(self, corpus):
        super(Decoder, self).__init__()
        self.tgt_emb = nn.Embedding(len(corpus.tgt_vocab), d_embedding)
        self.pos_emb = nn.Embedding.from_pretrained(get_sin_enc_table(corpus.tgt_len + 1, d_embedding), freeze=True)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])
    
    def forward(self, dec_inputs, enc_inputs, enc_outputs):
        # dec_inputs 的维度是 [batch_size, target_len]
        # enc_inputs 的维度是 [batch_size, source_len]
        # enc_outputs 的维度是 [batch_size, source_len, embedding_dim]
        
        # 创建位置索引，将词嵌入和位置嵌入相加
        pos_indices = torch.arange(1, dec_inputs.size(1) + 1).unsqueeze(0).to(dec_inputs)
        # dec_outputs -> [batch_size, target_len, embedding_dim]
        dec_outputs = self.tgt_emb(dec_inputs) + self.pos_emb(pos_indices)
        
        # 生成各种掩码
        # 填充掩码-多头自注意力
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs)
        # 后续掩码-多头自注意力
        dec_self_attn_subsequent_mask = get_attn_subsequent_mask(dec_inputs)
        # dec_self_attn_mask -> [batch_size, target_len, target_len]
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequent_mask), 0)
        
        # 填充掩码-编码器-解码器多头注意力
        # dec_enc_attn_mask -> [batch_size, target_size, source_size]
        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs)
        
        dec_self_attns, dec_enc_attns = [], []
        for layer in self.layers:
            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, 
                                                               dec_self_attn_mask, dec_enc_attn_mask)
            dec_self_attns.append(dec_self_attn)
            dec_enc_attns.append(dec_enc_attn)
        
        return dec_outputs, dec_self_attns, dec_enc_attns

### 组件10 Transformer
the final Transformer

In [81]:
# 定义Transformer模型
class Transformer(nn.Module):
    def __init__(self, corpus):
        super(Transformer, self).__init__()
        self.encoder = Encoder(corpus)
        self.decoder = Decoder(corpus)
        # 定义线性投影层，将解码器的输出转换为目标词汇表大小的概率分布
        self.projection = nn.Linear(d_embedding, len(corpus.tgt_vocab), bias=False)
    
    def forward(self, enc_inputs, dec_inputs):
        #------------------------- 维度信息 --------------------------------
        # enc_inputs -> [batch_size, source_seq_len]
        # dec_inputs 的维度是 [batch_size, target_seq_len]
        #-----------------------------------------------------------------
        
        # 将输入传递给编码器，得到编码器的输出和自注意力权重
        # enc_outputs -> [batch_size, source_len, embedding_dim]
        # enc_self_attns是一个list，里面的每个元素的形状为 -> [batch_size, n_heads, tgt_seq_len, src_seq_len]
        enc_outputs, enc_self_attns = self.encoder(enc_inputs)
        
        # 将编码器的输入和输出、解码器的输入传递给Decoder
        # dec_outputs(解码器层输出) -> [batch_size,  target_len, embedding_dim]
        # dec_self_attns和dec_enc_attns都为list，分别存储解码器自注意力和编码器-解码器注意力权重
        # dec_self_attn -> [batch_size, n_head, target_seq_len, target_seq_len]
        # dec_enc_attns -> [batch_size, n_head, target_seq_len, source_seq_len]
        dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs, enc_inputs, enc_outputs)
        
        # 将解码器输出传递给投影层，生成目标词汇白噢大小的概率分布
        # dec_logits -> [batch_size, target_seq_len, target_voc_size]
        dec_logits = self.projection(dec_outputs)
        
        return dec_logits, enc_self_attns, dec_self_attns, dec_enc_attns

## 一个简单的翻译任务
仍然使用小型翻译任务数据集，创建一个TranslationCorpus类，读入语料，自动整理语料库的字段，并提供批量数据

> 准备数据

In [82]:
sentences = [
    ['咖哥 喜欢 小冰', 'KaGe likes XiaoBing'],
    ['我 爱 学习 人工智能', 'I love studying AI'],
    ['深度学习 改变 世界', ' DL changed the world'],
    ['自然语言处理 很 强大', 'NLP is powerful'],
    ['神经网络 非常 复杂', 'Neural-networks are complex'] ]

In [83]:
from collections import Counter # 导入 Counter 类
# 定义 TranslationCorpus 类
class TranslationCorpus:
    def __init__(self, sentences):
        self.sentences = sentences
        # 计算源语言和目标语言的最大句子长度，并分别加 1 和 2 以容纳填充符和特殊符号
        self.src_len = max(len(sentence[0].split()) for sentence in sentences) + 1
        self.tgt_len = max(len(sentence[1].split()) for sentence in sentences) + 2
        # 创建源语言和目标语言的词汇表
        self.src_vocab, self.tgt_vocab = self.create_vocabularies()
        # 创建索引到单词的映射
        self.src_idx2word = {v: k for k, v in self.src_vocab.items()}
        self.tgt_idx2word = {v: k for k, v in self.tgt_vocab.items()}
    # 定义创建词汇表的函数
    def create_vocabularies(self):
        # 统计源语言和目标语言的单词频率
        src_counter = Counter(word for sentence in self.sentences for word in sentence[0].split())
        tgt_counter = Counter(word for sentence in self.sentences for word in sentence[1].split())        
        # 创建源语言和目标语言的词汇表，并为每个单词分配一个唯一的索引
        src_vocab = {'<pad>': 0, **{word: i+1 for i, word in enumerate(src_counter)}}
        tgt_vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2, 
                     **{word: i+3 for i, word in enumerate(tgt_counter)}}        
        return src_vocab, tgt_vocab
    # 定义创建批次数据的函数
    def make_batch(self, batch_size, test_batch=False):
        input_batch, output_batch, target_batch = [], [], []
        # 随机选择句子索引
        sentence_indices = torch.randperm(len(self.sentences))[:batch_size]
        for index in sentence_indices:
            src_sentence, tgt_sentence = self.sentences[index]
            # 将源语言和目标语言的句子转换为索引序列
            src_seq = [self.src_vocab[word] for word in src_sentence.split()]
            tgt_seq = [self.tgt_vocab['<sos>']] + [self.tgt_vocab[word] \
                         for word in tgt_sentence.split()] + [self.tgt_vocab['<eos>']]            
            # 对源语言和目标语言的序列进行填充
            src_seq += [self.src_vocab['<pad>']] * (self.src_len - len(src_seq))
            tgt_seq += [self.tgt_vocab['<pad>']] * (self.tgt_len - len(tgt_seq))            
            # 将处理好的序列添加到批次中
            input_batch.append(src_seq)
            output_batch.append([self.tgt_vocab['<sos>']] + ([self.tgt_vocab['<pad>']] * \
                                    (self.tgt_len - 2)) if test_batch else tgt_seq[:-1])
            target_batch.append(tgt_seq[1:])        
          # 将批次转换为 LongTensor 类型
        input_batch = torch.LongTensor(input_batch)
        output_batch = torch.LongTensor(output_batch)
        target_batch = torch.LongTensor(target_batch)            
        return input_batch, output_batch, target_batch
# 创建语料库类实例
corpus = TranslationCorpus(sentences)

> 模型训练

In [84]:
import torch
import torch.optim as optim

# 创建模型实例
model = Transformer(corpus)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
epoches = 5
for epoch in range(epoches):
    optimizer.zero_grad()
    enc_inputs, dec_inputs, target_batch = corpus.make_batch(batch_size)
    outputs, _, _, _ = model(enc_inputs, dec_inputs)
    loss = criterion(outputs.view(-1, len(corpus.tgt_vocab)), target_batch.view(-1))
    if (epoch + 1) % 1 == 0: # 打印损失
        print(f"Epoch: {epoch + 1:04d} cost = {loss:.6f}")
    loss.backward()# 反向传播        
    optimizer.step()# 更新参数

Epoch: 0001 cost = 3.010521
Epoch: 0002 cost = 2.734045
Epoch: 0003 cost = 2.422033
Epoch: 0004 cost = 2.023669
Epoch: 0005 cost = 1.304951


> 结果测试

In [89]:
# 创建一个大小为1的batch
enc_inputs, dec_inputs, target_batch = corpus.make_batch(batch_size=1, test_batch=True)
print("编码器输入 :", enc_inputs) # 打印编码器输入
print("解码器输入 :", dec_inputs) # 打印解码器输入
print("目标数据 :", target_batch) # 打印目标数据
predict, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs) # 用模型进行翻译
predict = predict.view(-1, len(corpus.tgt_vocab))
predict = predict.data.max(1, keepdim=True)[1]
res = [corpus.tgt_idx2word[idx.item()] for idx in predict.squeeze()]

input_sentence = ' '.join([corpus.src_idx2word[idx.item()] for idx in enc_inputs[0]])
print(input_sentence, '->', res) # 打印原始句子和翻译后的句子

编码器输入 : tensor([[14, 15, 16,  0,  0]])
解码器输入 : tensor([[1, 0, 0, 0, 0]])
目标数据 : tensor([[17, 18, 19,  2,  0]])
神经网络 非常 复杂 <pad> <pad> -> ['DL', 'DL', 'DL', '<eos>', '<eos>']
